In [1]:
import numpy as np 
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from itertools import chain
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from keras.metrics import categorical_crossentropy
import itertools
from sklearn.metrics import confusion_matrix

In [2]:
# Unzip Attack Folder to retrive subfolfers
from zipfile import ZipFile
file_name = "Attack_Data_Master.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Unizipped Attack File')

Unizipped Attack File


In [3]:
# Unzip Validation Folder to retrive subfolfers
from zipfile import ZipFile
file_name = "Validation_Data_Master.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Unizipped Validation File')

Unizipped Validation File


In [4]:
from pathlib import Path 
# list that holds attack vectors
train_attack_data = []

# Set file path ot the folder for iterations
input_dir = Path.cwd() / "Attack_Data_Master"
# store all the files ending with .txt in a list called files
files = list (input_dir.rglob("*.txt*"))
# Iterate over all the txt files and append the attack in the list train_attack data
for FILE in files:
  with open (FILE, 'r') as f:
    stringData = (f.read())
    string_split_data = stringData.split(" ")
    del (string_split_data[-1])
    train_attack_data.append(string_split_data)
    

for i in range(len(train_attack_data)):
  for j in range(len(train_attack_data[i])):
    train_attack_data[i][j] = int(train_attack_data[i][j])


In [5]:
from pathlib import Path 
# list that holds attack vectors
train_validation_data = []

# Set file path ot the folder for iterations
input_dir = Path.cwd() / "Validation_Data_Master"
# store all the files ending with .txt in a list called files
files = list (input_dir.rglob("*.txt*"))
# Iterate over all the txt files and append the attack in the list train_attack data
for FILE in files:
  with open (FILE, 'r') as f:
    stringData = (f.read())
    string_split_data = stringData.split(" ")
    del (string_split_data[-1])
    train_validation_data.append(string_split_data)
    

for i in range(len(train_validation_data)):
  for j in range(len(train_validation_data[i])):
    train_validation_data[i][j] = int(train_validation_data[i][j])

In [6]:
len(train_validation_data)

4372

In [7]:
# list holding normal training data
train_normal_data = [] 

files = 834 # maximum number of files for training data
main = "UTD-0000"
# iterate over all files
for i in range(1,files):
  back = str(i)
# appending each training vector to the list train_data.
  with open (main[:len(main)-len(back)] + back  + ".txt", 'r') as f:
    string_data = (f.read())
    string_split_data = string_data.split(" ")
    del (string_split_data[-1])
    train_normal_data.append(string_split_data)

for i in range(len(train_normal_data)):
  for j in range(len(train_normal_data[i])):
    train_normal_data[i][j] = int(train_normal_data[i][j])

In [8]:
#HOLD
# Appending normal and attack vectors to training sample
train_samples = []
train_labels = []  # 0 stands for normal behaviour, 1 stands for anomolous

for i in train_normal_data:
  train_samples.append(i)
  train_labels.append(0)  
  

for i in train_attack_data:
  train_samples.append(i)
  train_labels.append(1)


In [9]:
# Normalizing Data
vocab_size = 340 # Total unique number of system calls
for i in range (len(train_samples)):
  for j in range (len(train_samples[i])):
       train_samples[i][j] /= vocab_size

In [10]:
#NEW
x_train = []
y_train = []
result = [0,1]
#seq_y = tensorflow.keras.utils.to_categorical(result, num_classes = 2)

# Accepts a request of variable length, number of sequences to make, and 
# index according to its behaviour.
# This method splits the request in given number of system calls sequences and 
# tags with its respective label.
def setup_Input(req,anomaly_switch):

    if anomaly_switch == 0:
      for i in range(train_samples):
        x_train.append(train_samples[i])
        y_train.append(0)
    if anomaly_switch == 1:
        for i in range(train_samples):
            x_train.append()
            y_train.append(1)
    

    

In [12]:
# Normal Behaviour, -> INDEX [0]
# Attack  -> INDEX [1]

num_sequences = 15  # Number of sequences to divide the request in.

for i in range(len(train_samples)):
  if i > 832:
    setup_Input(train_samples[i],num_sequences,1)
  else:
    setup_Input(train_samples[i],num_sequences,0)

TypeError: setup_Input() takes 2 positional arguments but 3 were given

In [ ]:
# Converting x_train and y_train into np arrays
x_train = np.array(x_train)
y_train = np.array(y_train)
x_train, y_train = shuffle(x_train, y_train)

In [ ]:
# split test data into training and testing sets
x_train, x_test,y_train,y_test = train_test_split(x_train,y_train,test_size=0.2,random_state=4)

In [ ]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], n_features))
print(x_train.shape)

In [ ]:
model = Sequential()

model.add(LSTM(128, input_shape = (x_train.shape[1:]), return_sequences=False))
model.add(Dropout(0.2))

# model.add(LSTM(128))
# model.add(Dropout(0.2))

model.add(Dense(2,activation='softmax'))

opt = tf.keras.optimizers.Adam(learning_rate= 1e-2,decay=1e-6)

In [ ]:
# Compile the Model
model.compile(optimizer=opt, loss ='sparse_categorical_crossentropy', metrics= 'accuracy')

In [ ]:
history = model.fit(x_train,y_train, batch_size = 200, epochs=200)

In [ ]:
# Loss and Accuracy
score = model.evaluate(x_test,y_test, batch_size=128,)
model.metrics_names

In [ ]:
# Graph of the loss function 
plt.plot(history.history['loss'])
plt.show()

In [ ]:
# Predictions of the model on the test data
predictions = model.predict(x_test, batch_size = 128,verbose= 0)
rounded_predictions = np.argmax(predictions,axis=-1)

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_true = y_test, y_pred = rounded_predictions)

In [ ]:
def plot_confusion_matrix(cm, classes, normalize= False, title = "Confusion Matrix", cmap=plt.cm.Blues):
  plt.imshow(cm,interpolation='nearest', cmap = cmap)
  plt.title(title)
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes, rotation=45)
  plt.yticks(tick_marks, classes)

  if normalize:
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print("Normalized confusion matrix")
  else:
      print("Confusion Matrix, without normalization")
  
  print(cm)

  thresh = cm.max() / 2
  for i,j in itertools.product(range(cm.shape[0]),range(cm.shape[1])):
    plt.text(j,i,cm[i,j],
      horizontalalignment = 'center',
      color = 'white' if cm[i,j] > thresh else 'black')
    

  plt.tight_layout()
  plt.ylabel("True Label")
  plt.xlabel("Predicted Label")

In [ ]:
cm_plot_labels = ['Attack_sys_call', 'Normal_sys_call']
plot_confusion_matrix(cm = cm, classes = cm_plot_labels, title = 'Confusion Matrix')

In [ ]:
###############################################################################################################

In [ ]:
x_validate = []

# Accepts a request of variable length, number of sequences.
# This method splits the request in given number of system calls sequences 
def split_validation_request(req,n_features):
  length = len(req)
  length = int(length/n_features)
  start = 0;
  end = n_features;


  for i in range(length):
    seq_x = req[start:end]
    x_validate.append(seq_x)
    end += n_features;
    start = end - n_features;

In [ ]:
# Normalizing Validation Data
for i in range (len(train_validation_data)):
  for j in range (len(train_validation_data[i])):
       train_validation_data[i][j] /= vocab_size

In [ ]:
for i in range(len(train_validation_data)):
    split_validation_request(train_validation_data[i],num_sequences)

In [ ]:
v_predictions = model.predict(x_validate, batch_size = 128,verbose= 0)
rounded_v_predictions = np.argmax(v_predictions,axis=-1)

In [ ]:
# open the file using open() function
file = open("predictions.txt", 'w')
correct = 0
incorrect = 0
   
for i in range(len(rounded_v_predictions)):
    if(rounded_v_predictions[i] == 1):
        incorrect+=1
    else:
        correct+=1
    file.write(str(rounded_v_predictions[i])) 
    file.write(" ")
# closing the file
file.close()
 

In [ ]:
print(incorrect/(correct+incorrect))